In [ ]:
# Changing the working directory
import os
os.chdir('../')

In [1]:
import pandas as pd
from utils import calldb
from skydb.sheets import *

In [2]:
stmt = """
    SELECT academic_enrollments.user_id, academic_enrollments.term, 
    academic_enrollments.section_id, tp_sections.teacher_id, tp_teachers.first_name,
    tp_teachers.last_name
    FROM academic_enrollments
    JOIN tp_sections ON academic_enrollments.section_id = tp_sections.id
    JOIN tp_teachers ON tp_sections.teacher_id = tp_teachers.id
"""

In [3]:
results = calldb(stmt)

In [4]:
df = pd.DataFrame(
    results,
    columns=['user_id', 'term', 'section_id', 'teacher_id', 'teacher_first', 'teacher_last']
)

In [5]:
teacherLoadReport = (
    df.groupby(['teacher_id', 'term'])
    .count()
    .rename(columns={
     'user_id':'n'
    })
    .drop(['section_id', 'teacher_first', 'teacher_last'], axis=1)
    .sort_values('n')
    .reset_index()
    .merge(
        df[['teacher_id', 'teacher_first', 'teacher_last']]
        .drop_duplicates()
    )
)

In [6]:
# updateSpreadsheet(
#     teacherLoadReport, 
#     tab_name='Spring',
#     sheet_id="1KBvQBKxw4zlyeQbASRpn1vBkm9m86y0Tvd9Ykr9pq08"
# )

In [7]:
from utils import getEnrollments

In [8]:
fallEnrollments = getEnrollments(False)

In [9]:
fallTeacherLoadReport = (
    fallEnrollments[
        (fallEnrollments
         .term_name
         .str
         .split(
             ' ', 
             expand=True
         )[0] == "Fall"
        )
    ]
    .groupby('teacher_id')
    .count()
    .rename(columns={
     'user_id':'n'
    })['n']
    .sort_values()
    .reset_index()
    .merge(
        fallEnrollments[['teacher_id', 'teacher_first', 'teacher_last']]
        .drop_duplicates()
    )
)

In [10]:
# updateSpreadsheet(
#     fallTeacherLoadReport, 
#     tab_name='Fall',
#     sheet_id="1KBvQBKxw4zlyeQbASRpn1vBkm9m86y0Tvd9Ykr9pq08"
# )

In [11]:
updateSpreadsheet(
    fallTeacherLoadReport,
    tab_name="New Test",
    sheet_id="1KBvQBKxw4zlyeQbASRpn1vBkm9m86y0Tvd9Ykr9pq08"
)

In [12]:
import gspread
gc = gspread.service_account()
sheet = gc.open_by_key('1KBvQBKxw4zlyeQbASRpn1vBkm9m86y0Tvd9Ykr9pq08')
worksheet = sheet.worksheet('New Test')

In [13]:
# worksheet.clear()

In [14]:
# worksheet.columns_auto_resize(0, 100)